In [7]:
import  pandas as pd, polars as pl
from    pathlib import Path
from    toolz import first

from    latent_action_models.data_exploration import cache

ACTION_PARQUET  = Path(first(cache.__path__)) / 'gta_frames.parquet'
EVENT_PARQUET   = Path(first(cache.__path__)) / 'gta_events.parquet'

In [ ]:
proc_df = pl.read_parquet(ACTION_PARQUET)
proc_df.head()

# TODO Mouse deltas should be normalized somehow, but 0 to 1 might not cut it?
# TODO Why are some actions null instead of being 0s? I think it's cause we should be 
# aggregating on the groupby better? nulls should not exist here - i notice that when dx/dy is 0 buttons are 0
# but when dx/dy is a number the rest are null.
# TODO pos_x: -51; pos_y: <-- why is this a column!!!
# TODO 0; wheelmouse2 <-- why is this also a column..?

# TODO Once all the above is sorted, cancel out antagonistic actions and filter out non WASD/SHIFT/CTRL actions
# TODO Then, check to see what's overrepresented and what is not, perhaps use some BPE for keypresses.

frame_idx,dx,dy,wheel,u,a,7,f,h,5,equal,l,o,g,e,j,space,c,m,w,z,control,2,mouse3,down,t,mouse1,i,bracketright,capslock,left,alt,up,right,b,comma,r,p,8,n,k,mouse4,minus,q,6,1,s,v,mouse2,9,shift,3,0; wheelmouse2,0,y,x,mouse5,tab,backslash,d,4,pos_x: -51; pos_y:,video_path,t_sec
i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,f64
56,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"""/mnt/data/shahbuland/video-pro…",1.866667
71,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"""/mnt/data/shahbuland/video-pro…",2.366667
75,-113.0,-504.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""/mnt/data/shahbuland/video-pro…",2.5
76,-707.0,-2121.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""/mnt/data/shahbuland/video-pro…",2.533333
77,-680.0,-1868.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""/mnt/data/shahbuland/video-pro…",2.566667


In [8]:
raw_df = pl.read_parquet(EVENT_PARQUET)
raw_df.head()

video_metadata.video_path,video_metadata.start,video_metadata.end,video_metadata.fps,video_metadata.codec,video_metadata.exc,action_data.origin_csv,action_data.origin_vid,action_data.lineno,action_data.raw,action_data.t_sec,action_data.event_type,action_data.keys_active,action_data.key_pressed,action_data.key_pressed_dir,action_data.transition,action_data.dx,action_data.dy,action_data.wheel,action_data.exc
str,f64,f64,f64,str,str,str,str,i64,str,f64,str,list[str],str,str,null,f64,f64,f64,null
"""/mnt/data/shahbuland/video-pro…",0.0,110153.0,60.0,"""hevc""",null,"""/mnt/data/shahbuland/video-pro…","""/mnt/data/shahbuland/video-pro…",0,"""1.655,W,W down """,1.655,"""key""","[""w""]","""w""","""down""",null,null,null,null,null
"""/mnt/data/shahbuland/video-pro…",0.0,110153.0,60.0,"""hevc""",null,"""/mnt/data/shahbuland/video-pro…","""/mnt/data/shahbuland/video-pro…",1,"""2.155,W,W down """,2.155,"""key""","[""w""]","""w""","""down""",null,null,null,null,null
"""/mnt/data/shahbuland/video-pro…",0.0,110153.0,60.0,"""hevc""",null,"""/mnt/data/shahbuland/video-pro…","""/mnt/data/shahbuland/video-pro…",2,"""2.187,W,W down """,2.187,"""key""","[""w""]","""w""","""down""",null,null,null,null,null
"""/mnt/data/shahbuland/video-pro…",0.0,110153.0,60.0,"""hevc""",null,"""/mnt/data/shahbuland/video-pro…","""/mnt/data/shahbuland/video-pro…",3,"""2.234,W,W down """,2.234,"""key""","[""w""]","""w""","""down""",null,null,null,null,null
"""/mnt/data/shahbuland/video-pro…",0.0,110153.0,60.0,"""hevc""",null,"""/mnt/data/shahbuland/video-pro…","""/mnt/data/shahbuland/video-pro…",4,"""2.265,W,W down """,2.265,"""key""","[""w""]","""w""","""down""",null,null,null,null,null
